In [1]:
import os
# os.getcwd()
os.chdir("/localdisk4/panwla/Projects/park_vlm/Kangaroo")
os.getcwd()

'/localdisk4/panwla/Projects/park_vlm/Kangaroo'

In [2]:
import json
import os
import boto3
import yaml
# from google.colab import drive

# # Mount Google Drive to access API keys
# drive.mount('/content/drive', force_remount=True)

# Load API keys from file
file_path = './.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)

hf_read_api_key = api_keys['HUGGINGFACE']['HF_READ_API_KEY']

from huggingface_hub import login
login(hf_read_api_key)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /localdisk4/panwla/huggingface_cache/token
Login successful


In [3]:
import os

# Set environment variables based on your bashrc settings
os.environ["JUPYTER_DATA_DIR"] = "/localdisk4/panwla/jupyter_data"
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/localdisk4/panwla/huggingface_cache/transformers"
os.environ["HF_DATASETS_CACHE"] = "/localdisk4/panwla/huggingface_cache/datasets"
os.environ["TORCH_HOME"] = "/localdisk4/panwla/torch"
os.environ["XDG_CACHE_HOME"] = "/localdisk4/panwla_cache/.cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"
os.environ["TEMP"] = "/localdisk4/panwla/tmp"
os.environ["TMP"] = "/localdisk4/panwla/tmp"

# Create directories if they don't exist
directories = [
    os.environ["JUPYTER_DATA_DIR"],
    os.environ["HF_HOME"],
    os.environ["TRANSFORMERS_CACHE"],
    os.environ["HF_DATASETS_CACHE"],
    os.environ["TORCH_HOME"],
    os.environ["XDG_CACHE_HOME"],
    os.environ["TMPDIR"],
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)


In [4]:

facial_expressions_query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''


Apparent_diff_task ='''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video. 

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: “Yes” or “No”.

Answer: 

'''


BG_and_lighting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Mention if the background is overloaded (i.e., too many things), or the lighting condition is inappropriate (i.e., too dark or overlit). Otherwise, just output “normal background”.

Answer: 

'''


Blink_rate_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Was there anything abnormal about the person’s eye blink rate? For example, they may not be blinking at all, or they may have reduced or increased blink rate compared to a normal person. If there is nothing abnormal, output “normal blinking”.

Answer: 

'''


Camera_position_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: How far is the camera? A good position of the camera would be when the upper half of the subject's body remains visible, while the lower half is not captured in the frame. If the upper body is only partly visible, the camera is too close. If the lower body is also visible, the camera is too far.

Answer: 

'''


Coherence_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Is the subject coherent in what they are speaking? Are they delivering an easy to understand story, or are they deterring a lot from the central topic and delivering an unorganized speech?

Answer: 

'''


Comp_task_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Please indicate whether the subject was able to follow the instructions while completing the task. If the subject was doing something differently, please describe.

Answer: 


'''

Lip_parting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate the extent of lips parting when the subject is not saying anything (i.e., always/most of the times/sometimes/very few times/never).

Answer: 

'''


Masked_faceExp_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate which of the following are true for the subject: (i) The individual's face appears blank and emotionless, even when they are trying to express an emotion. (ii) The expression is weak or asymmetrical, and the individual has difficulty holding an expression (e.g., smile) for an extended period. Also mention if you observe other facial expression abnormalities.

Answer: 

'''

Observations_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Document any abnormal signs observed in body parts other than the face. This includes, but is not limited to, tremors in the hands, involuntary shaking or rhythmic movements of the upper or lower extremities, stiffness or rigidity in the limbs, reduced arm swing while speaking, or any signs of bradykinesia (slowness of movement). Additionally, note any abnormal postures, difficulty in maintaining balance, or other motor irregularities that may be indicative of Parkinson’s disease.

Answer: 


'''

Overall_app_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Provide a brief description of the subject's perceived state of mind, noting whether they appear energetic, exhausted, calm, confused, or exhibit any other relevant emotional or cognitive cues.

Answer: 


'''


Other_people_present_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate whether any other individuals were present in the background. If so, provide a brief description (e.g., "An older male is visible in the background"). Conclude with a final answer: "Yes" or "No".

Answer: 
 

'''


Comp_sent_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>


Question: Indicate whether the subject is using complex sentences that are difficult to understand. Conclude with a final answer: "Yes" or "No".

Answer: 


'''


Other_BP_visible_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate if any body part critical for this task is partially visible (or invisible). For example, was the subject wearing a mask that may have obstructed important visual information? Or did the face go out of frame while the subject was completing the task?

Answer: 


'''

In [5]:
import os
import pandas as pd

# === Setup file paths ===
cayla_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Cayla/cayla_df_with_transcriptions.csv"
nami_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Clinical/Nami/nami_common_final.csv"
natalia_files = "/localdisk4/panwla/Projects/park_vlm/Annotations/Clinical/Natalia/natalia_common_final.csv"

video_path = "/localdisk4/panwla/Projects/park_vlm/Videos/Videos"

# === Load video references ===
cayla_df = pd.read_csv(cayla_files)
nami_df = pd.read_csv(nami_files)
natalia_df = pd.read_csv(natalia_files)



# Identify prompt + video columns
prompt_columns = list(cayla_df.columns[0:13])  # first 13 columns
video_column = ['video']

# Subset Nami and Natalia using same columns
nami_df = nami_df.loc[:, prompt_columns + video_column]
natalia_df = natalia_df.loc[:, prompt_columns + video_column]

# List available videos
available_videos = os.listdir(video_path)

# === Empty DataFrame to collect results ===
columns = ["Video_file", "Q_ID", "Prompt_name", "Model_Name", "Cayla_Resp", "Nami_Resp", "Natalia_Resp"]
output = pd.DataFrame(columns=columns)
output

,Video_file,Q_ID,Prompt_name,Model_Name,Cayla_Resp,Nami_Resp,Natalia_Resp


In [6]:
cayla_df.columns

Index(['Apparent difficulty completing the task', 'Background and lighting',
       'Blink rate', 'Camera Position', 'Coherence',
       'Compliance with tasks instructions',
       'Lips parting when the mouth is at rest', 'Masked facies',
       'Observations of other body parts not being directly assessed',
       'Overall appearance', 'Presence of other persons',
       'Usage of complex sentence', 'Visibility of significant body parts',
       'video', 'Transcription'],
      dtype='object')

In [7]:

# === Store prompts in a dictionary ===
prompt_dict = {
    "facial_expressions_query": facial_expressions_query,
    "BG_and_lighting_query": BG_and_lighting_query,
    "Blink_rate_query": Blink_rate_query,
    "Camera_position_query": Camera_position_query,
    "Coherence_query": Coherence_query,
    "Comp_task_query": Comp_task_query,
    "Lip_parting_query": Lip_parting_query,
    "Masked_faceExp_query": Masked_faceExp_query,
    "Observations_query": Observations_query,
    "Overall_app_query": Overall_app_query,
    "Other_people_present_query": Other_people_present_query,
    "Other_BP_visible_query": Other_BP_visible_query,
    
}



In [8]:
import os
import pandas as pd
import copy
import torch
import gc
import sys
import numpy as np
import warnings
from PIL import Image

# Import Hugging Face Transformers for VideoLLaMA3
from transformers import AutoModelForCausalLM, AutoProcessor

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Set PyTorch memory allocation configuration
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
warnings.filterwarnings("ignore")


def load_videollama3_model(model_name="DAMO-NLP-SG/VideoLLaMA3-7B", 
                        #    device="cuda"
                           ):
    """
    Load the VideoLLaMA3 model and processor from Hugging Face.
    """
    print(f"Loading VideoLLaMA3 model: {model_name}")
    # Clean up any previous CUDA usage
    torch.cuda.empty_cache()
    gc.collect()
    
    model = AutoModelForCausalLM.from_pretrained(
         model_name,
         trust_remote_code=True,
         device_map=None,
         torch_dtype=torch.bfloat16,
         attn_implementation="flash_attention_2",
    ).to("cuda")  
    processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
    model.eval()
    print("VideoLLaMA3 model loaded successfully")
    return model, processor


def analyze_video_with_videollama3(
    video_path, 
    prompt_text, 
    model,
    processor, 
    max_new_tokens=128, 
    fps=1, 
    max_frames=128, 
    # device="cuda"
):
    """
    Process a video using VideoLLaMA3 by constructing a conversation input that includes the video.
    """
    try:
        # Clear GPU cache before processing
        torch.cuda.empty_cache()
        gc.collect()
        
        # pick device the model lives on
        if hasattr(model, "hf_device_map"):
            first_device = next(iter(model.hf_device_map.values()))
        else:
            first_device = next(model.parameters()).device     # works in single‑GPU mode

        
        # Construct the conversation input with video and text components.
        conversation = [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": {"video_path": video_path, "fps": fps, "max_frames": max_frames}},
                    {"type": "text", "text": prompt_text},
                ]
            },
        ]
        
        # Process the conversation to create inputs for the model.
        inputs = processor(conversation=conversation, return_tensors="pt")

        # move everything to the model’s device without changing dtype
        inputs = {k: (v.to(first_device) if torch.is_tensor(v) else v)
                for k, v in inputs.items()}

        # now, *optionally* down‑cast only the heavy video tensor
        if "pixel_values" in inputs:
            inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)

        with torch.inference_mode():
            output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
        
        response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
        
        # Clear GPU memory after processing
        torch.cuda.empty_cache()
        gc.collect()
        
        return response
        
    except Exception as e:
        print(f"⚠️ Error analyzing video with VideoLLaMA3: {e}")
        torch.cuda.empty_cache()
        gc.collect()
        return f"ERROR: {str(e)}"


def process_all_prompts(input_df, prompt_dict, video_path, model_components, nami_df=None, natalia_df=None, batch_size=1):
    """
    Process the list of prompts on the provided videos using the VideoLLaMA3 model.
    This function searches for video files in the provided folder, applies any prompt-specific
    modifications (e.g. inserting a transcription), and then processes each video for all prompts.
    """
    model, processor = model_components
    available_videos = [f for f in os.listdir(video_path) if f.endswith(".mp4")]
    
    # List of prompts that need transcription insertion.
    transcription_prompts = ["Coherence_query", "Comp_task_query", "Comp_sent_query"]
    
    # Define output columns for the results DataFrame.
    columns = ["Video_file", "Q_ID", "Prompt_name", "Model_Name", "Output", "Cayla_Resp", "Nami_Resp", "Natalia_Resp"]
    output = pd.DataFrame(columns=columns)
    
    # Process videos in batches to manage memory.
    for batch_start in range(0, len(input_df), batch_size):
        batch_end = min(batch_start + batch_size, len(input_df))
        print(f"Processing batch {batch_start//batch_size + 1} (rows {batch_start} to {batch_end-1})")
        
        for idx, row in input_df.iloc[batch_start:batch_end].iterrows():
            video_name = row.get("video", None)
            if not isinstance(video_name, str) or not video_name:
                continue
                
            transcription = row.get("Transcription", "")
            # Locate matching video file in the folder.
            matched_file = next((f for f in available_videos
                                 if video_name[-25:] in f), None)
        
            if not matched_file:
                print(f"❌ No matching video for {video_name}")
                continue

            full_video_path = os.path.join(video_path, matched_file)
            prompt_columns = list(input_df.columns[:13])
            
            for q_id, (prompt_name, prompt_text) in enumerate(prompt_dict.items(), start=1):
                processed_prompt = prompt_text
                if prompt_name in transcription_prompts and transcription:
                    processed_prompt = processed_prompt.replace("<TRANSCRIPTION_OUTPUT>", transcription)
                    print(f"🔄 Modified prompt {prompt_name} with transcription")
                
                # Retrieve previous responses if available.
                cayla_resp = row[prompt_columns[q_id - 1]] if q_id - 1 < len(prompt_columns) else ""
                
                nami_resp = ""
                if nami_df is not None:
                    nami_row = nami_df[nami_df["video"] == video_name]
                    if not nami_row.empty and q_id - 1 < len(prompt_columns):
                        nami_resp = nami_row[prompt_columns[q_id - 1]].values[0]
                
                natalia_resp = ""
                if natalia_df is not None:
                    natalia_row = natalia_df[natalia_df["video"] == video_name]
                    if not natalia_row.empty and q_id - 1 < len(prompt_columns):
                        natalia_resp = natalia_row[prompt_columns[q_id - 1]].values[0]
                
                # Process video using VideoLLaMA3.
                try:
                    response = analyze_video_with_videollama3(
                        full_video_path,
                        processed_prompt,
                        model,
                        processor,
                        max_new_tokens=128,  # You can adjust this value as needed
                        fps=1,
                        max_frames=128,
                        # device="cuda"
                    )
                except RuntimeError as e:
                    print(f"💥 Critical error processing {prompt_name}, attempting recovery: {e}")
                    torch.cuda.empty_cache()
                    gc.collect()
                    
                    
                # Append results to the output DataFrame.
                new_row = {
                    "Video_file": matched_file,
                    "Q_ID": q_id,
                    "Prompt_name": prompt_name,
                    "Model_Name": "VideoLLaMA3",
                    "Output": response,
                    "Cayla_Resp": cayla_resp,
                    "Nami_Resp": nami_resp,
                    "Natalia_Resp": natalia_resp
                }
                
                output = pd.concat([output, pd.DataFrame([new_row])], ignore_index=True)
                output.to_csv("videollama3_results_partial.csv", index=False)
            
            # Clean GPU memory after processing each video.
            torch.cuda.empty_cache()
            gc.collect()
        
        # Save intermediate results after each batch.
        output.to_csv(f"videollama3_results_batch_{batch_start//batch_size + 1}.csv", index=False)
        
    return output


# Load VideoLLaMA3 model and processor
model_components = load_videollama3_model(
    model_name="DAMO-NLP-SG/VideoLLaMA3-7B",
)

# Process all prompts using VideoLLaMA3
results_df = process_all_prompts(
    input_df=cayla_df,
    prompt_dict=prompt_dict,
    video_path=video_path,
    model_components=model_components,
    nami_df=nami_df,
    natalia_df=natalia_df,
    batch_size=1  # Process one video at a time to optimize memory usage
)

# Save the final results
# results_df.to_csv("videollama3_results_final.csv", index=False)
print("Analysis complete! Results saved to videollama3_results_final.csv")


/localdisk4/panwla/conda_envs/env3.10/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-7B:
- configuration_videollama3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading VideoLLaMA3 model: DAMO-NLP-SG/VideoLLaMA3-7B


A new version of the following files was downloaded from https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-7B:
- modeling_videollama3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

A new version of the following files was downloaded from https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-7B:
- processing_videollama3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


VideoLLaMA3 model loaded successfully
Processing batch 1 (rows 0 to 0)
⚠️ Error analyzing video with VideoLLaMA3: expand(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got Tensor"
⚠️ Error analyzing video with VideoLLaMA3: expand(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got Tensor"
⚠️ Error analyzing video with VideoLLaMA3: expand(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got Tensor"
⚠️ Error analyzing video with VideoLLaMA3: expand(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got Tensor"
🔄 Modified prompt Coherence_query with transcription
⚠️ Error analyzing video with VideoLLaMA3: expand(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got Tensor"
🔄 Modified prompt Comp_task_query with transcription
⚠️ Error analyzing

KeyboardInterrupt: 